In [1]:
import json
import torch
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yaml
from networks import HRNet_w32_256x192
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [2]:
!ls

cocoapi			    dataloader.py  models	README.md
cuda-keyring_1.1-1_all.deb  metrics	   networks.py	scripts
data			    metrics.ipynb  __pycache__	trainer.py


In [3]:
with open('models/HRNet/configs/w32_256x192_adam_lr1e-3.yaml', "r") as file:
    config = yaml.safe_load(file)
    print(config)

{'AUTO_RESUME': True, 'CUDNN': {'BENCHMARK': True, 'DETERMINISTIC': False, 'ENABLED': True}, 'DATA_DIR': '', 'GPUS': '(0,1,2,3)', 'OUTPUT_DIR': 'output', 'LOG_DIR': 'log', 'WORKERS': 24, 'PRINT_FREQ': 100, 'DATASET': {'COLOR_RGB': True, 'DATASET': 'coco', 'DATA_FORMAT': 'jpg', 'FLIP': True, 'NUM_JOINTS_HALF_BODY': 8, 'PROB_HALF_BODY': 0.3, 'ROOT': 'data/coco/', 'ROT_FACTOR': 45, 'SCALE_FACTOR': 0.35, 'TEST_SET': 'val2017', 'TRAIN_SET': 'train2017'}, 'MODEL': {'INIT_WEIGHTS': True, 'NAME': 'pose_hrnet', 'NUM_JOINTS': 17, 'PRETRAINED': 'models/HRNet/paths/pose_hrnet_w32_256x192.pth', 'TARGET_TYPE': 'gaussian', 'IMAGE_SIZE': [192, 256], 'HEATMAP_SIZE': [48, 64], 'SIGMA': 2, 'EXTRA': {'PRETRAINED_LAYERS': ['conv1', 'bn1', 'conv2', 'bn2', 'layer1', 'transition1', 'stage2', 'transition2', 'stage3', 'transition3', 'stage4'], 'FINAL_CONV_KERNEL': 1, 'STAGE2': {'NUM_MODULES': 1, 'NUM_BRANCHES': 2, 'BLOCK': 'BASIC', 'NUM_BLOCKS': [4, 4], 'NUM_CHANNELS': [32, 64], 'FUSE_METHOD': 'SUM'}, 'STAGE3':

In [4]:
extra = config['MODEL']['EXTRA']
extra['FINAL_CONV_KERNEL']

1

### Load Models

In [2]:
# test load HRNet
HRNet = HRNet_w32_256x192()

In [ ]:
# load models
models = [('placeholder'), models.alexnet()]

metrics = {
    "Placeholder1": {"FLOPs": 4.1e9, "params": 23e6, "MACs": 2.5e9},
}

### FLOPS

In [ ]:
INPUT_SIZE = (3, 244, 244)

In [ ]:
# loop over all models
for model_name, model in models:
    batch_size = 1
    input_shape = (batch_size, INPUT_SIZE)
    flops, macs, params = calculate_flops(model=model, 
                                        input_shape=input_shape,
                                        output_as_string=True,
                                        output_precision=4)
    metrics[model_name] = {
        "FLOPs": flops, 
        "params": macs, 
        "MACs": params
    }
    print("Model: %s, FLOPs:%s   MACs:%s   Params:%s \n" %(model_name, flops, macs, params))

### Accuracy

In [ ]:
# define test_loader
test_loader = None
# define loss function
criterion = None

In [ ]:
def test(model, test_loader):
    model.eval() # set model in evaluation mode
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(test_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (datapoints, labels) in enumerate(test_loader):

        ### Move data to device (ideally GPU)
        datapoints      = datapoints.to(device)
        labels    = labels.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(datapoints)
            ### Loss Calculation
            loss    = criterion(logits, labels)

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == labels).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del datapoints, labels, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(test_loader)
    tacc    /= len(test_loader)

    return tloss, tacc

In [ ]:
for model_name, model in models:
    loss, acc = test(model=model, test_loader=test_loader)
    metrics[model_name] = {
        "loss": loss, 
        "acc": acc, 
    }
    print("Model: %s, Loss: %s, Acc: %s \n" %(model_name, loss, acc))

### Plot

In [ ]:
df = pd.DataFrame.from_dict(metrics, orient="index")
df.reset_index(inplace=True)
df.rename(columns={"index": "model_name"}, inplace=True)

print(df)

# save to csv file
df.to_csv("IDL_11785_Project/metrics/metrics_data.csv", index=False)

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(data=df, x="name", y="FLOPs")
plt.title("Comparison of FLOPs Across Models")
plt.xlabel("Model")
plt.ylabel("FLOPs")
plt.xticks(rotation=45)
plt.show()

plt.savefig("IDL_11785_Project/metrics/plots/FLOPs_comparison.png", dpi=300, transparent=True, bbox_inches='tight')